In [11]:
# Step 1: Load Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Step 2: Load Data
data = pd.read_csv("data_for_AI_project_100k.csv", encoding='latin1')
data = data.drop(columns=['RISK_TYPE', 'DISBURSED_AMOUNT'], errors='ignore')

# Step 3: Handle Categorical Columns
categorical_cols = [
    "EMI_PERSON_GENDER", "LOCALITY", "STATE_NAME", "BRANCH_NAME",
    "REGION_NAME", "TOPUP_LOAN_ORNOT", "CUSTOMER_PROFILE", "ENDUSE"
]
label_encoders = {}
for col in categorical_cols:
    data[col] = data[col].fillna("Other")
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Step 4: Fill Nulls for Numerical Columns
numerical_cols = [
    "AGE", "BUSINESS_INCOME", "BUSINESSVINTAGE_WORK_EXP", "CIBIL_SCORE",
    "LOAN_AMOUNT", "LOAN_BALANCE", "EMI_AMOUNT", "INT_RATE", "TENURE"
]
for col in numerical_cols:
    data[col] = data[col].fillna(data[col].median())

# Step 5: Simulate Binary Risk Label (for training)
data['risk_binary'] = np.where(data['CIBIL_SCORE'] < 600, 1, 0)

# Step 6: Feature Selection & Scaling
drop_cols = ['SCHEME_NAME', 'CUST_ID', 'CUST_NAME', 'LOAN_ID', 'EMI_CYCLE_DT', 'risk']
features = data.drop(columns=drop_cols + ['risk_binary'], errors='ignore')
features = features.fillna(features.median(numeric_only=True))
features = features.select_dtypes(include=[np.number])
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
target = data['risk_binary']

# Step 7: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Step 8: Build Neural Network
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping to avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, epochs=20, batch_size=256, validation_split=0.2, callbacks=[early_stop])

# Step 9: Evaluate Model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Step 10: Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster'] = kmeans.fit_predict(features_scaled)

# Step 11: Convert Probabilities to Risk Class
def classify_risk(prob):
    if prob <= 0.3:
        return 0  # Low risk
    elif prob <= 0.7:
        return 1  # Medium risk
    else:
        return 2  # High risk

# Step 12: Recommend Action
def recommend_action(row):
    if row['risk'] == 2:
        return "Legal Action"
    elif row['risk'] == 1:
        return "Visit Customer" if row['cluster'] == 0 else "Call Customer" if row['cluster'] == 1 else "Send SMS Reminder"
    else:
        return "Send SMS Reminder" if row['cluster'] == 0 else "Call Customer" if row['cluster'] == 1 else "No Action Needed"

# Predict risk scores for entire dataset
risk_probs = model.predict(features_scaled).flatten()
data['risk'] = [classify_risk(p) for p in risk_probs]
data['recommended_action'] = data.apply(recommend_action, axis=1)

# Step 13: Prediction for New Customer
def safe_transform(le, value):
    if value in le.classes_:
        return le.transform([value])[0]
    if 'Other' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'Other')
    return le.transform(['Other'])[0]

def predict_new_customer_risk(new_data):
    for col in categorical_cols:
        new_data[col] = new_data[col].apply(lambda x: safe_transform(label_encoders[col], x))
    for col in numerical_cols:
        new_data[col] = new_data[col].fillna(data[col].median())
    new_data = new_data.fillna(0)
    new_features = new_data[features.columns.tolist()].select_dtypes(include=[np.number])
    new_scaled = scaler.transform(new_features)
    risk_prob = model.predict(new_scaled).flatten()
    predicted_risk = [classify_risk(p) for p in risk_prob]
    cluster = kmeans.predict(new_scaled)
    new_data['risk'] = predicted_risk
    new_data['cluster'] = cluster
    new_data['recommended_action'] = new_data.apply(recommend_action, axis=1)
    return new_data

# Example New Customer
new_customer = pd.DataFrame([{
    'AGE': 30,
    'BUSINESS_INCOME': 300000,
    'BUSINESSVINTAGE_WORK_EXP': 5,
    'CIBIL_SCORE': 550,
    'LOAN_AMOUNT': 1000000,
    'LOAN_BALANCE': 200000,
    'EMI_AMOUNT': 15000,
    'INT_RATE': 10,
    'TENURE': 12,
    'EMI_PERSON_GENDER': 'Male',
    'LOCALITY': 'Urban',
    'STATE_NAME': 'Kerala',
    'BRANCH_NAME': 'Thrissur',
    'REGION_NAME': 'South',
    'TOPUP_LOAN_ORNOT': 'No',
    'CUSTOMER_PROFILE': 'Business',
    'ENDUSE': 'Business Expansion'
}])

result = predict_new_customer_risk(new_customer)
print(result[['risk', 'cluster', 'recommended_action']])


Epoch 1/20


c:\Users\420317\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8416 - loss: 0.3417 - val_accuracy: 0.9900 - val_loss: 0.0444
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9744 - loss: 0.0633 - val_accuracy: 0.9918 - val_loss: 0.0284
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9820 - loss: 0.0421 - val_accuracy: 0.9937 - val_loss: 0.0191
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9886 - loss: 0.0272 - val_accuracy: 0.9956 - val_loss: 0.0146
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9911 - loss: 0.0216 - val_accuracy: 0.9971 - val_loss: 0.0112
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9926 - loss: 0.0172 - val_accuracy: 0.9958 - val_loss: 0.0110
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9932 - loss: 0.0157 - val_accuracy: 0.9953 - val_loss: 0.0107
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9948 - loss: 0.0124 - val_accuracy: 0.9979 - val_

In [8]:
print(data.head(10))

             SCHEME_NAME         CUST_ID           CUST_NAME  \
0        Loan to Teacher   7701851957678       Barbara Rojas   
1  Consumer Durable Loan   6535111823658  Kathleen Gutierrez   
2   Small Scale Industry  89014701188405        Michael Rich   
3                   MSME  71480168980801    Donald Hernandez   
4  Loan to Food Industry  40674270093776          Karen Ross   
5  Loan to Food Industry  13111346573990         James Davis   
6   Small Scale Industry  94480936982774      Kristina Mejia   
7                   MSME  18869596451817       Timothy Brady   
8  Loan to Food Industry  55299948861398        Michael Rice   
9   Small Scale Industry  83955969942012     Robert Thompson   

   EMI_PERSON_GENDER  AGE  LOCALITY           LOAN_ID  BUSINESS_INCOME  \
0                  2   31         1  5815226887277624           704840   
1                  1   62         0  4069747216195695           184367   
2                  1   57         1  2299370678301755           289706   

In [10]:
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.99785


In [4]:
data.to_csv("cleaned_AI_project_data.csv", index=False)
print("✅ Cleaned dataset saved as cleaned_AI_project_data.csv")

✅ Cleaned dataset saved as cleaned_AI_project_data.csv


In [9]:
import pickle

# Save objects for Streamlit
model.save("risk_model.h5")
pickle.dump(scaler, open("scaler.pkl", "wb"))
pickle.dump(kmeans, open("kmeans.pkl", "wb"))
pickle.dump(label_encoders, open("label_encoders.pkl", "wb"))
pickle.dump(features.columns.tolist(), open("features_list.pkl", "wb"))
